This tutorial describes how to evaluate new rules applicable to the derivatives data.

In [1]:
from arelle import ModelManager, Cntlr, ModelFormulaObject, ModelXbrl, ViewFileFormulae, XbrlConst, ViewFileRenderedGrid
from arelle import RenderingEvaluator 

In [2]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import join, isfile
import re
from src import Evaluator
import logging
import data_patterns

In [3]:
DECIMALS = 0
RULES_PATH = join('..', 'solvency2-rules')
INSTANCES_DATA_PATH = join('..', 'data', 'instances')
FILENAME_DATAPOINTS = 'QRS.csv'
RESULTS_PATH = join('..', 'results')
DATA_PATH = join('..', 'data')
logging.basicConfig(filename = join(RESULTS_PATH, 'rules.log'),level = logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

### Import rules

We start with importing the new rules applicable to the derivatives data. One set of rules is applicable to template S2.08.01.01 (Open derivatives). This template consists of both S2.08.01.01.01 (Information on positions held) and S2.08.01.01.02 (Information on derivatives). The other set of rules is only applicable to S2.08.01.01.02 (Information on derivatives).

In [4]:
dfr = pd.read_excel(join(RULES_PATH,'S2_08_01_01.xlsx'), index_col='index')
dfr2 = pd.read_excel(join(RULES_PATH,'S2_08_01_01_02.xlsx'), index_col='index')

#Capitalize row-column references:
column_replace = set([column for sublist in [row for row in dfr['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
column_replace = set([column for sublist in [row for row in dfr2['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr2.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)   

### Import templates

Next we import the reporting data. In the tutorial 'Convert Solvency 2 XBRL-instances to CSV, HTML and pickles' the XBRL-instances are converted to pickle files per template. The pickle files are written to the data/instances folder. 

We create two dataframes with reporting data. For the set of rules that is applicable to template S2.08.01.01 (Open derivatives) we merge S2.08.01.01.01 (Information on positions held) and S2.08.01.01.02 (Information on derivatives). The other set of rules is only applicable to S2.08.01.01.02 (Information on derivatives).

For the sake of simplicity we only import the Quarterly Solvency II reporting Solo (QRS) templates.


In [5]:
df1 = pd.read_pickle(join(join(INSTANCES_DATA_PATH,'qrs_240_instance'),'S.08.01.01.01.pickle')).reset_index()
df2 = pd.read_pickle(join(join(INSTANCES_DATA_PATH,'qrs_240_instance'), 'S.08.01.01.02.pickle')).reset_index()
df = pd.merge(df1,df2,how='inner', left_on=['entity','period','S.08.01.01.01,C0040'], right_on=['entity','period','S.08.01.01.02,C0040']).set_index(['entity', 'period', 'S.08.01.01.01,C0040'])
df2 = df2.set_index(['entity', 'period', 'S.08.01.01.02,C0040'])

In [6]:
df

,,,index_x,"S.08.01.01.01,C0440","S.08.01.01.01,C0070","S.08.01.01.01,C0090","S.08.01.01.01,C0060","S.08.01.01.01,C0080","S.08.01.01.01,C0110","S.08.01.01.01,C0120","S.08.01.01.01,C0130","S.08.01.01.01,C0140",...,"S.08.01.01.02,C0330","S.08.01.01.02,C0340","S.08.01.01.02,C0360","S.08.01.01.02,C0370","S.08.01.01.02,C0380","S.08.01.01.02,C0390","S.08.01.01.02,C0400","S.08.01.01.02,C0410","S.08.01.01.02,C0420","S.08.01.01.02,C0430"
entity,period,"S.08.01.01.01,C0040",,,,,,,,,,,,,,,,,,,,,
0LFF1WMNTWG5PTIYYI38,2019-12-31,ISIN/EZ3104183600,0,1,1,ISIN/CE6248278617,Ring Fenced Funds,Unit-linked or index-linked,"Efficient portfolio management, other than “Ma...",411729.0,6.597989e+08,FX-FL [open],...,BNP Paribas SA,LEI/R0MUWSFPU8MPRO8K5P83,fh tj lu,EUR,USA2,20,Adverse fall in value of the underlying refere...,0,0,2013-10-05 00:00:00


### Evaluate rules

Now we are ready to evaluate the rules. First, we construct a PatternMiner-object with the data-patterns package using the rules dataframe. Second, we use the analyze-function to get the results of the rules. We do this for both set of rules separately. 

In [10]:
miner = data_patterns.PatternMiner(df_patterns=dfr)
results = miner.analyze(df)
results

result_type  \
0LFF1WMNTWG5PTIYYI38 2019-12-31 ISIN/EZ3104183600         True   
                                ISIN/EZ3104183600         True   

                                                        pattern_id  cluster  \
0LFF1WMNTWG5PTIYYI38 2019-12-31 ISIN/EZ3104183600  Contract size 2        0   
                                ISIN/EZ3104183600   Buyer/seller 2        0   

                                                   support  exceptions  \
0LFF1WMNTWG5PTIYYI38 2019-12-31 ISIN/EZ3104183600        1           0   
                                ISIN/EZ3104183600        1           0   

                                                   confidence  \
0LFF1WMNTWG5PTIYYI38 2019-12-31 ISIN/EZ3104183600         1.0   
                                ISIN/EZ3104183600         1.0   

                                                                                         pattern_def  \
0LFF1WMNTWG5PTIYYI38 2019-12-31 ISIN/EZ3104183600  IF {"S.08.01.01.02,C0380"} = "USA2" THEN {"S.0...   
                                ISIN/EZ3104183600  IF {"S.08.01.01.02,C0380"} = "USA2" THEN {"S.0...   

                                                  P values      Q values  
0LFF1WMNTWG5PTIYYI38 2019-12-31 ISIN/EZ3104183600     USA2        760703  
                                ISIN/EZ3104183600     USA2  FX-FL [open]

In [9]:
miner = data_patterns.PatternMiner(df_patterns=dfr2)
results2 = miner.analyze(df2)
results2

result_type  \
0LFF1WMNTWG5PTIYYI38 2019-12-31 ISIN/EZ3104183600         True   
                                ISIN/EZ3104183600         True   
                                ISIN/EZ3104183600         True   
                                ISIN/EZ3104183600        False   
                                ISIN/EZ3104183600        False   

                                                                pattern_id  \
0LFF1WMNTWG5PTIYYI38 2019-12-31 ISIN/EZ3104183600               Currency 1   
                                ISIN/EZ3104183600  Counterparty group code   
                                ISIN/EZ3104183600          External rating   
                                ISIN/EZ3104183600               Currency 3   
                                ISIN/EZ3104183600      Credit quality step   

                                                   cluster  support  \
0LFF1WMNTWG5PTIYYI38 2019-12-31 ISIN/EZ3104183600        0        1   
                                ISIN/EZ3104183600        0        1   
                                ISIN/EZ3104183600        0        1   
                                ISIN/EZ3104183600        0        0   
                                ISIN/EZ3104183600        0        0   

                                                   exceptions  confidence  \
0LFF1WMNTWG5PTIYYI38 2019-12-31 ISIN/EZ3104183600           0         1.0   
                                ISIN/EZ3104183600           0         1.0   
                                ISIN/EZ3104183600           0         1.0   
                                ISIN/EZ3104183600           1         0.0   
                                ISIN/EZ3104183600           1         0.0   

                                                                                         pattern_def  \
0LFF1WMNTWG5PTIYYI38 2019-12-31 ISIN/EZ3104183600  IF ({"S.08.01.01.02,C0380"} = "USA2") THEN (({...   
                                ISIN/EZ3104183600  IF {"S.08.01.01.02,C0330"} = "BNP Paribas SA" ...   
                                ISIN/EZ3104183600  IF (({"S.08.01.01.02,C0260"}= "The Goldman Sac...   
                                ISIN/EZ3104183600  IF {"S.08.01.01.02,C0380"} = "USA2" THEN {"S.0...   
                                ISIN/EZ3104183600  IF (({"S.08.01.01.02,C0260"}= "The Goldman Sac...   

                                                                             P values  \
0LFF1WMNTWG5PTIYYI38 2019-12-31 ISIN/EZ3104183600                                USA2   
                                ISIN/EZ3104183600                      BNP Paribas SA   
                                ISIN/EZ3104183600  [The Goldman Sachs Group, Inc., 0]   
                                ISIN/EZ3104183600                                USA2   
                                ISIN/EZ3104183600  [The Goldman Sachs Group, Inc., 0]   

                                                                   Q values  
0LFF1WMNTWG5PTIYYI38 2019-12-31 ISIN/EZ3104183600               [0, 0, EUR]  
                                ISIN/EZ3104183600  LEI/R0MUWSFPU8MPRO8K5P83  
                                ISIN/EZ3104183600                      BBB+  
                                ISIN/EZ3104183600                       EUR  
                                ISIN/EZ3104183600     Credit quality step 1